# DeepEval: Evaluating LangChain Agents with LLM-as-Judge

**A Complete Guide to Evaluating AI Agents**

[![DeepEval](https://img.shields.io/badge/DeepEval-Latest-purple.svg)](https://docs.confident-ai.com/)
[![LangChain](https://img.shields.io/badge/LangChain-0.2+-green.svg)](https://python.langchain.com/)
[![Python 3.11+](https://img.shields.io/badge/python-3.11+-blue.svg)](https://www.python.org/downloads/)

---

## Overview

This notebook demonstrates how to:
1. Build a **Loan Advisor Agent** using [LangChain](https://python.langchain.com/)
2. Evaluate it with [DeepEval](https://docs.confident-ai.com/) LLM-as-Judge metrics
3. Follow best practices for Golden → Dataset → Evaluation workflow

### Key Concepts

| Concept | Description |
|---------|-------------|
| **LangChain Agent** | ReAct agent with tools for loan calculations |
| **DeepEval Golden** | Test template (input + expected behavior) |
| **LLMTestCase** | Actual test with agent outputs |
| **Metrics** | Answer Relevancy, Faithfulness, Hallucination |

---

## 1. Installation

In [ ]:
# Install required packages
# !pip install -q deepeval langchain langchain-openai langgraph pandas python-dotenv

In [ ]:
import os
import json
from typing import List, Dict, Any, Annotated

# =============================================================================
# API Key Configuration
# Supports: Local (.env), Kaggle Secrets, Google Colab Secrets
# =============================================================================

def setup_api_key():
    """Setup OpenAI API key from various sources."""
    
    # 1. Check if already set in environment
    if os.getenv("OPENAI_API_KEY"):
        print("✓ API key loaded from environment")
        return True
    
    # 2. Try Kaggle Secrets
    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        os.environ["OPENAI_API_KEY"] = user_secrets.get_secret("OPENAI_API_KEY")
        print("✓ API key loaded from Kaggle Secrets")
        return True
    except:
        pass
    
    # 3. Try Google Colab Secrets
    try:
        from google.colab import userdata
        os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
        print("✓ API key loaded from Colab Secrets")
        return True
    except:
        pass
    
    # 4. Try .env file (local development)
    try:
        from dotenv import load_dotenv
        load_dotenv()
        if os.getenv("OPENAI_API_KEY"):
            print("✓ API key loaded from .env file")
            return True
    except:
        pass
    
    return False

# Setup API key
if not setup_api_key():
    print("⚠️  OPENAI_API_KEY not found!")
    print("   Please set it using one of these methods:")
    print("   - Kaggle: Add to Secrets (Add-ons → Secrets)")
    print("   - Colab: Add to Secrets (🔑 icon in sidebar)")
    print("   - Local: Create .env file with OPENAI_API_KEY=sk-...")
    raise ValueError("OPENAI_API_KEY is required")

print("\nEnvironment configured!")

## 2. Define Loan Advisor Tools

Create tools for loan calculations using LangChain's `@tool` decorator.

In [ ]:
from langchain_core.tools import tool

@tool
def calculate_loan_payment(
    loan_amount: float,
    annual_interest_rate: float,
    loan_term_months: int
) -> str:
    """
    Calculate monthly loan payment using standard amortization formula.
    
    Args:
        loan_amount: The principal loan amount in dollars
        annual_interest_rate: Annual interest rate as decimal (e.g., 0.05 for 5%)
        loan_term_months: Loan term in months
    
    Returns:
        JSON string with payment details
    """
    # Monthly interest rate
    monthly_rate = annual_interest_rate / 12
    
    # Calculate monthly payment using amortization formula
    if monthly_rate == 0:
        monthly_payment = loan_amount / loan_term_months
    else:
        monthly_payment = loan_amount * (
            monthly_rate * (1 + monthly_rate) ** loan_term_months
        ) / ((1 + monthly_rate) ** loan_term_months - 1)
    
    total_payment = monthly_payment * loan_term_months
    total_interest = total_payment - loan_amount
    
    result = {
        "monthly_payment": round(monthly_payment, 2),
        "total_payment": round(total_payment, 2),
        "total_interest": round(total_interest, 2),
        "loan_amount": loan_amount,
        "annual_interest_rate": annual_interest_rate,
        "loan_term_months": loan_term_months
    }
    return json.dumps(result)


@tool
def check_loan_eligibility(
    age: int,
    monthly_income: float,
    credit_score: int,
    requested_loan_amount: float,
    employment_status: str
) -> str:
    """
    Check if applicant is eligible for a loan based on criteria.
    
    Args:
        age: Applicant's age in years
        monthly_income: Monthly income in dollars
        credit_score: Credit score (300-850)
        requested_loan_amount: Requested loan amount in dollars
        employment_status: Employment status (full_time, part_time, self_employed, unemployed)
    
    Returns:
        JSON string with eligibility result
    """
    reasons = []
    is_eligible = True
    
    # Age check (18-65)
    if age < 18 or age > 65:
        is_eligible = False
        reasons.append(f"Age {age} outside eligible range (18-65)")
    else:
        reasons.append("Age requirement met")
    
    # Income check (minimum $3000)
    if monthly_income < 3000:
        is_eligible = False
        reasons.append(f"Income ${monthly_income} below minimum ($3000)")
    else:
        reasons.append("Income requirement met")
    
    # Credit score check
    if credit_score < 600:
        is_eligible = False
        credit_rating = "Poor"
        reasons.append(f"Credit score {credit_score} below minimum (600)")
    elif credit_score < 670:
        credit_rating = "Fair"
        reasons.append("Credit score acceptable")
    elif credit_score < 740:
        credit_rating = "Good"
        reasons.append("Credit score good")
    else:
        credit_rating = "Excellent"
        reasons.append("Credit score excellent")
    
    # Employment check
    if employment_status == "unemployed":
        is_eligible = False
        reasons.append("Unemployed applicants not eligible")
    else:
        reasons.append(f"Employment status ({employment_status}) acceptable")
    
    # DTI ratio estimate
    estimated_monthly_payment = requested_loan_amount / 36  # Rough estimate
    dti_ratio = estimated_monthly_payment / monthly_income
    
    if dti_ratio > 0.5:
        is_eligible = False
        reasons.append(f"DTI ratio {dti_ratio:.1%} exceeds 50% limit")
    
    result = {
        "is_eligible": is_eligible,
        "credit_rating": credit_rating,
        "dti_ratio": round(dti_ratio, 3),
        "reasons": reasons,
        "max_recommended_loan": round(monthly_income * 0.5 * 36, 2)
    }
    return json.dumps(result)


@tool
def check_loan_affordability(
    loan_amount: float,
    annual_interest_rate: float,
    loan_term_months: int,
    monthly_income: float,
    existing_monthly_debt: float
) -> str:
    """
    Check if a loan is affordable based on debt-to-income ratio.
    
    Args:
        loan_amount: Loan amount in dollars
        annual_interest_rate: Annual interest rate as decimal
        loan_term_months: Loan term in months
        monthly_income: Monthly income in dollars
        existing_monthly_debt: Existing monthly debt payments
    
    Returns:
        JSON string with affordability analysis
    """
    # Calculate monthly payment
    monthly_rate = annual_interest_rate / 12
    if monthly_rate == 0:
        monthly_payment = loan_amount / loan_term_months
    else:
        monthly_payment = loan_amount * (
            monthly_rate * (1 + monthly_rate) ** loan_term_months
        ) / ((1 + monthly_rate) ** loan_term_months - 1)
    
    # Calculate DTI ratios
    current_dti = existing_monthly_debt / monthly_income
    new_dti = (existing_monthly_debt + monthly_payment) / monthly_income
    
    is_affordable = new_dti <= 0.5
    
    result = {
        "is_affordable": is_affordable,
        "monthly_payment": round(monthly_payment, 2),
        "current_dti": round(current_dti, 3),
        "new_dti": round(new_dti, 3),
        "max_dti": 0.5,
        "total_monthly_debt": round(existing_monthly_debt + monthly_payment, 2),
        "remaining_income": round(monthly_income - existing_monthly_debt - monthly_payment, 2)
    }
    return json.dumps(result)


# Collect all tools
tools = [calculate_loan_payment, check_loan_eligibility, check_loan_affordability]

print(f"Defined {len(tools)} tools:")
for t in tools:
    print(f"  - {t.name}")

## 3. Create LangChain ReAct Agent

Using LangGraph's `create_react_agent` for a production-ready agent.

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

# System prompt for loan advisor
system_prompt = """You are a helpful Loan Advisor assistant. 
You help users with loan calculations, eligibility checks, and affordability analysis.
Always use the provided tools to give accurate information.
Be clear and concise in your responses."""

# Create ReAct agent using LangGraph
# Note: Use 'prompt' parameter (state_modifier is deprecated)
agent = create_react_agent(
    llm,
    tools,
    prompt=system_prompt
)

print("LangChain ReAct Agent created!")

## 4. Agent Runner with Context Extraction

This class runs queries through the agent and extracts:
- `actual_output`: Agent's final response
- `tools_called`: Which tools were invoked
- `retrieval_context`: Tool outputs (ground truth for evaluation)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

class LangChainAgentRunner:
    """Runs queries through LangChain agent and extracts evaluation data."""
    
    def __init__(self, agent):
        self.agent = agent
    
    def run(self, query: str) -> Dict[str, Any]:
        """
        Run query through agent and extract results.
        
        Returns:
            Dict with actual_output, tools_called, retrieval_context
        """
        # Invoke agent
        result = self.agent.invoke({
            "messages": [HumanMessage(content=query)]
        })
        
        messages = result["messages"]
        
        # Extract final output (last AI message)
        actual_output = ""
        for msg in reversed(messages):
            if isinstance(msg, AIMessage) and msg.content:
                actual_output = msg.content
                break
        
        # Extract tool calls and results
        tools_called = []
        retrieval_context = []
        
        for msg in messages:
            # Tool calls from AI messages
            if isinstance(msg, AIMessage) and hasattr(msg, 'tool_calls'):
                for tc in msg.tool_calls:
                    if isinstance(tc, dict):
                        tools_called.append(tc.get('name', ''))
            
            # Tool results
            if isinstance(msg, ToolMessage):
                retrieval_context.append(msg.content)
        
        return {
            "actual_output": actual_output,
            "tools_called": tools_called,
            "retrieval_context": retrieval_context
        }


# Create runner
runner = LangChainAgentRunner(agent)
print("Agent runner ready!")

## 5. Test the Agent

In [ ]:
# Quick test
test_query = "Calculate my monthly payment for a $50,000 loan at 5% interest for 36 months."

print(f"Query: {test_query}")
print("\nRunning agent...")

result = runner.run(test_query)

print(f"\nTools called: {result['tools_called']}")
print(f"\nAgent response:\n{result['actual_output']}")
print(f"\nRetrieval context: {result['retrieval_context'][:1]}...")  # First tool result

## 6. Define Goldens (Test Templates)

Following [DeepEval best practices](https://www.confident-ai.com/docs/llm-evaluation/core-concepts/test-cases-goldens-datasets).

In [ ]:
GOLDENS = [
    {
        "id": "loan_calculation",
        "input": "Calculate my monthly payment for a $50,000 loan at 5% annual interest for 36 months.",
        "expected_tools": ["calculate_loan_payment"],
        "expected_keywords": ["1,498", "1498", "monthly"],
        "complexity": "simple"
    },
    {
        "id": "eligibility_check",
        "input": "Check my loan eligibility: I'm 25 years old, monthly income $6000, credit score 720, requesting $30,000 loan. I work full-time.",
        "expected_tools": ["check_loan_eligibility"],
        "expected_keywords": ["eligible", "720", "good"],
        "complexity": "simple"
    },
    {
        "id": "affordability_check",
        "input": "Can I afford a $40,000 loan at 6% interest for 48 months? My monthly income is $5,500 and I have $500 in existing debt.",
        "expected_tools": ["check_loan_affordability"],
        "expected_keywords": ["afford", "DTI", "debt"],
        "complexity": "moderate"
    },
    {
        "id": "low_income_edge_case",
        "input": "I earn $2000/month with $300 existing debt. Can I afford a $50,000 loan at 7% for 60 months?",
        "expected_tools": ["check_loan_affordability"],
        "expected_keywords": ["not", "afford", "exceed"],
        "complexity": "edge_case"
    }
]

print(f"Defined {len(GOLDENS)} test cases:")
for g in GOLDENS:
    print(f"  [{g['complexity']}] {g['id']}")

## 7. Run Agent on All Test Cases

Execute each golden through the agent to get actual outputs.

In [ ]:
from deepeval.dataset import EvaluationDataset, Golden

print("Running agent on all test cases...")
print("=" * 60)

goldens_with_output = []

for g in GOLDENS:
    print(f"\nRunning: {g['id']}")
    
    # Run through agent
    result = runner.run(g["input"])
    
    print(f"  Tools: {result['tools_called']}")
    print(f"  Output: {result['actual_output'][:100]}...")
    
    # Create Golden with actual output
    golden = Golden(
        input=g["input"],
        actual_output=result["actual_output"],
        retrieval_context=result["retrieval_context"],
        additional_metadata={
            "test_id": g["id"],
            "complexity": g["complexity"],
            "expected_tools": g["expected_tools"],
            "actual_tools": result["tools_called"],
            "expected_keywords": g["expected_keywords"]
        }
    )
    goldens_with_output.append(golden)

# Create dataset
dataset = EvaluationDataset(goldens=goldens_with_output)

print("\n" + "=" * 60)
print(f"Created dataset with {len(dataset.goldens)} test cases")

## 8. Define DeepEval Metrics

In [ ]:
from deepeval import evaluate
from deepeval.metrics import (
    AnswerRelevancyMetric,
    FaithfulnessMetric,
    HallucinationMetric,
    GEval,
)
from deepeval.test_case import LLMTestCase, LLMTestCaseParams

EVAL_MODEL = "gpt-4o-mini"

# Standard metrics
metrics = [
    AnswerRelevancyMetric(threshold=0.7, model=EVAL_MODEL),
    FaithfulnessMetric(threshold=0.7, model=EVAL_MODEL),
    HallucinationMetric(threshold=0.5, model=EVAL_MODEL),
]

# Custom metric for financial advice
financial_metric = GEval(
    name="Financial Accuracy",
    criteria="""Evaluate the financial accuracy of the response:
    1. Are calculations mathematically correct?
    2. Are financial terms used appropriately?
    3. Is the advice financially sound?""",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    threshold=0.7,
    model=EVAL_MODEL
)

all_metrics = metrics + [financial_metric]

print(f"Configured {len(all_metrics)} metrics:")
for m in all_metrics:
    name = m.name if hasattr(m, 'name') else m.__class__.__name__
    print(f"  - {name} (threshold: {m.threshold})")

## 9. Convert Goldens to Test Cases

In [ ]:
# Convert to LLMTestCase for evaluation
test_cases = [
    LLMTestCase(
        input=g.input,
        actual_output=g.actual_output,
        context=g.retrieval_context,           # For Hallucination
        retrieval_context=g.retrieval_context,  # For Faithfulness
    )
    for g in dataset.goldens
]

print(f"Created {len(test_cases)} LLMTestCase objects")

## 10. Run Evaluation

In [ ]:
print("Running DeepEval evaluation...")
print("=" * 60)

results = evaluate(
    test_cases=test_cases,
    metrics=all_metrics,
)

print("\n" + "=" * 60)
print("Evaluation Complete!")

## 11. Analyze Results

In [ ]:
import pandas as pd

results_data = []

print("\n" + "=" * 60)
print("DETAILED RESULTS")
print("=" * 60)

for i, (tc, golden) in enumerate(zip(test_cases, dataset.goldens)):
    meta = golden.additional_metadata
    test_id = meta["test_id"]
    complexity = meta["complexity"]
    
    print(f"\n[{complexity.upper()}] {test_id}")
    print("-" * 40)
    
    row = {"test_id": test_id, "complexity": complexity}
    
    for metric in all_metrics:
        metric.measure(tc)
        name = metric.name if hasattr(metric, 'name') else metric.__class__.__name__.replace("Metric", "")
        status = "PASS" if metric.is_successful() else "FAIL"
        print(f"  {name}: {metric.score:.2f} ({status})")
        
        row[name] = metric.score
        row[f"{name}_pass"] = metric.is_successful()
    
    results_data.append(row)

df = pd.DataFrame(results_data)

In [ ]:
# Summary statistics
print("\n" + "=" * 60)
print("SUMMARY")
print("=" * 60)

score_cols = [c for c in df.columns if not c.endswith('_pass') and c not in ['test_id', 'complexity']]

print("\nAverage Scores:")
for col in score_cols:
    print(f"  {col}: {df[col].mean():.2f}")

print("\nPass Rates:")
pass_cols = [c for c in df.columns if c.endswith('_pass')]
for col in pass_cols:
    print(f"  {col.replace('_pass', '')}: {df[col].mean():.1%}")

overall = df[pass_cols].all(axis=1).mean()
print(f"\nOverall Pass Rate: {overall:.1%}")

## 12. Tool Call Validation

In [ ]:
print("\n" + "=" * 60)
print("TOOL CALL VALIDATION")
print("=" * 60)

tool_accuracy = []

for golden in dataset.goldens:
    meta = golden.additional_metadata
    expected = set(meta["expected_tools"])
    actual = set(meta["actual_tools"])
    
    match = expected.issubset(actual)
    status = "PASS" if match else "FAIL"
    
    print(f"\n{meta['test_id']}: {status}")
    print(f"  Expected: {list(expected)}")
    print(f"  Actual:   {list(actual)}")
    
    tool_accuracy.append(match)

print(f"\nTool Accuracy: {sum(tool_accuracy)/len(tool_accuracy):.1%}")

In [ ]:
# Final results table
print("\n" + "=" * 60)
print("FINAL RESULTS TABLE")
print("=" * 60)
print(df.to_string(index=False))

## 13. Key Takeaways

### LangChain + DeepEval Integration

1. **Build Agent**: Use LangGraph's `create_react_agent` with custom tools
2. **Extract Context**: Tool outputs become `retrieval_context` for evaluation
3. **Evaluate**: Use DeepEval metrics (Relevancy, Faithfulness, Hallucination)

### Best Practices

| Practice | Description |
|----------|-------------|
| **Golden → TestCase** | Define templates, generate outputs at eval time |
| **Context = Tool Outputs** | Use tool results as ground truth |
| **Multiple Metrics** | Combine standard + custom (G-Eval) metrics |
| **Validate Tools** | Check correct tools were called |

---

### Resources

- [LangChain Agents](https://docs.langchain.com/oss/python/langchain/agents)
- [LangGraph ReAct Agent](https://langchain-ai.github.io/langgraph/how-tos/react-agent-from-scratch/)
- [DeepEval Documentation](https://docs.confident-ai.com/)
- [DeepEval Datasets](https://deepeval.com/docs/evaluation-datasets)
- [G-Eval Custom Metrics](https://deepeval.com/docs/metrics-llm-evals)